In [192]:
import GPy
import csv
import numpy as np
import scipy.stats
import sklearn.model_selection

In [193]:
file_name='./hehe.csv'
all_time = np.genfromtxt(file_name,usecols=(0), delimiter=",",dtype=None)
pos = np.genfromtxt(file_name, usecols=(1,2), delimiter=",")
macaddress = np.genfromtxt(file_name, usecols=(5,7,9),delimiter=",",dtype=str)
strength = np.genfromtxt(file_name, usecols=(6,8,10),delimiter=",")
timestamp, indices = np.unique(all_time, return_index=True)

In [196]:
pos.max(axis=0)

array([0.78773751, 0.92057979])

In [198]:
pos.min(axis=0)

array([-1.16252942, -1.7529892 ])

In [194]:
train_pos, test_pos, train_strength, test_strength = sklearn.model_selection.train_test_split(pos, strength)


In [189]:
train_pos

(659, 2)

In [165]:
kernel = GPy.kern.RBF(input_dim=2,variance=1.0, lengthscale=None)
m = []
for i in range(3):
    m.append(GPy.models.GPRegression(train_pos, train_strength[:,i].reshape(-1,1),kernel))
    #m[i].constrain_positive('.*rbf_variance')
    #m[i].Gaussian_noise.constrain_positive() 
    m[i].randomize()
    m[i].optimize(messages=True)


HBox(children=(VBox(children=(IntProgress(value=0, max=1000), HTML(value=''))), Box(children=(HTML(value=''),))))

HBox(children=(VBox(children=(IntProgress(value=0, max=1000), HTML(value=''))), Box(children=(HTML(value=''),))))

HBox(children=(VBox(children=(IntProgress(value=0, max=1000), HTML(value=''))), Box(children=(HTML(value=''),))))

In [166]:

xg1 =np.linspace(-2,1)
xg2 =np.linspace(-2,1)

test_data = np.array(list(itertools.product(test_x,test_y)))

test_X = np.zeros((xg1.size * xg2.size,2))
for i,x1 in enumerate(xg1):
    for j,x2 in enumerate(xg2):
        test_X[i+xg1.size*j,:] = [x1,x2]
    
test_X.shape

(2500, 2)

In [167]:
mean = []
variance = []
for i in range(3):
    mean.append(m[i].predict(test_X)[0])
    variance.append(m[i].predict(test_X)[1])

variance

[array([[-2247080.25611904],
        [-2195895.20786924],
        [-2144167.68922752],
        ...,
        [-1863966.77332462],
        [-1912547.50273862],
        [-1961337.13614487]]), array([[-81040.20414649],
        [-80193.69629541],
        [-79366.75511893],
        ...,
        [-88102.17240345],
        [-88045.82128621],
        [-88026.28205575]]), array([[24.56194234],
        [23.76135223],
        [23.04622686],
        ...,
        [18.73291013],
        [18.84563223],
        [19.00536199]])]

In [172]:
prob = np.ndarray((test_strength.shape[0], test_X.shape[0],3))
for j in range(3):
    for i in range(prob.shape[0]):
        std = variance[j]**2**0.25
        prob[i] = scipy.stats.norm(mean[j], std).pdf(test_strength[i,j].reshape(1,-1))
    # [#grid points, #samples in test set]
prob.shape

 /Users/Abby/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning:invalid value encountered in power
 /Users/Abby/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1653: RuntimeWarning:invalid value encountered in greater
 /Users/Abby/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:876: RuntimeWarning:invalid value encountered in greater_equal
 /Users/Abby/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:876: RuntimeWarning:invalid value encountered in less_equal
 /Users/Abby/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1654: RuntimeWarning:invalid value encountered in greater


(453, 2500, 3)

In [180]:
predict_pos = test_X[np.argmax(np.average(prob, axis=2), axis=1)]

In [183]:
sse = ((predict_pos - test_pos)**2).sum()

In [184]:
sse

578.2673801526389

In [191]:
a = m[0].to_dict()

{'X': [[-1.0665096895078603, -1.2569100568141773],
  [0.5353882668115918, 0.5470533592801967],
  [0.08317044337413175, -0.11441455718726633],
  [0.52272047196003, 0.5272659020816352],
  [0.01753855023708625, -0.10459672828489441],
  [-0.3996772535596346, -1.7529891973018863],
  [0.5484430628796798, 0.5735832741428634],
  [-0.8784193155385377, -0.7202029301404678],
  [-0.925611410814032, -0.26863374100140613],
  [-0.3996772535596346, -1.7529891973018863],
  [0.15704504711077505, -0.010645911319803133],
  [-0.1570651293156674, -0.06763203790710912],
  [-0.5209048113935364, 0.007121949277101396],
  [-0.9175986587482031, -1.5680414051456586],
  [-0.3996772535596346, -1.7529891973018863],
  [-0.9290387288797265, -0.4545690091115925],
  [-0.8784193155385377, -0.7202029301404678],
  [0.11004041834981472, -0.0820132642731396],
  [-0.6359212661362547, -1.6992828549550187],
  [-0.8784193155385377, -0.7202029301404678],
  [-1.044252083833594, -1.31166588512001],
  [-1.162529417496575, -0.92374665

In [ ]:
from_dict.